## This notebook aims to scrape all historical race results PDFs of AQU, BEL and SAR in 2019. 
The PDFs are store in the shared Google drive: https://drive.google.com/drive/u/0/folders/1cND78Y9CmdMm5B_v0tWPFGSolIG5q-6h. 

In [3]:
#!pip install pdfplumber

import pdfplumber
import os
import numpy as np
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 3.3 MB/s 
     |████████████████████████████████| 3.2 MB 6.6 MB/s 
     |████████████████████████████████| 142 kB 51.6 MB/s 
     |████████████████████████████████| 5.6 MB 29.4 MB/s 
     |████████████████████████████████| 4.1 MB 29.5 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [6]:
drive_path = '/content/drive/Shareddrives/ANLY590_GROUP_4_PROJECT'

In [7]:
path1 = drive_path + "/data/historical_result_pdf"

# Get all the data under the dir.
file_name_list = os.listdir(path1)

# Column names.
title = ["Track","Date","Race","LastRaced","Pgm","HorseName(Jockey)","Wgt","M/E","PP","Start","0.25","0.5","0.75","Str","Fin","Odds","Comments"]

data_list=[]

for pa in file_name_list:

    # Get different track names.
    path2 = drive_path + "/data/historical_result_pdf/{}".format(pa)
    file_name_list2 = os.listdir(path2)

    for p in file_name_list2:
      
        # Access single pdf file.
        path = drive_path + "/data/historical_result_pdf/{}/{}".format(pa,p)
        pdf_mt = pdfplumber.open(path)
        page = pdf_mt.pages[0]

        # Extract the text.
        text_data=page.extract_text()
        text_data=list(text_data.split('\n'))
        
        # A indicator for teh lines to keep.
        a=0
        da=[]

        # Find the lines to keep.
        for data in text_data:
            if data.find("FractionalTimes:")!=-1:
                a = 0
            if a == 1:
                da.append(list(data.split(' ')))
            if data.find("LastRaced Pgm HorseName(Jockey) WgtM/E PP Start")!=-1:
                a = 1

        # Keep the race and date of the pdf.
        date = list(text_data[0].split('-'))
        race = date[-1].replace("Race",'')
        date = date[1]

        # Partition the data
        list2=['»','½','¶(cid:4)']
        for i in range(len(da)):
            da[i][-2]=da[i][-2].replace('*','')
            if da[i][3].find("L")!=-1 or da[i][3].find("b") != -1:
                for f in list2:
                    if da[i][3].find(f) != -1:
                        b = list(da[i][3].split(f))
                        da[i][3] = b[0]
                        da[i].insert(4, b[-1])
            for f in list2:
                da[i][3] = da[i][3].replace(f, '')
            if len(da[i])==13:
                da[i].insert(9,'---')
            if len(da[i])==12:
                da[i].insert(8,'---')
            da[i].insert(0, race)
            da[i].insert(0, date)
            da[i].insert(0, pa)
            
        for data in da:
            if len(data) != 17:
                print(path, "something's wrong")
                print(da)
                print(data)
                continue
            data_list.append(data)

# Create the dataframe.
data = np.array(data_list).transpose()
dict2 = {}
for i in range(len(data)):
    dict2[title[i]]=data[i]

data = pd.DataFrame(dict2)
data.to_csv(drive_path + "/data/raw/historical_results.csv",index=False)

/content/drive/Shareddrives/ANLY590_GROUP_4_PROJECT/data/historical_result_pdf/BEL-BELMONT PARK/05-12-2019-3.pdf something's wrong
[['BEL-BELMONT PARK', 'May12,2019', '3', '20Apr194LRL6', '3', 'Slimey(Carmouche,Kendrick)', '118', 'Lf', '2', '3', '21/2', '---', '39', '11/2', '131/2', '4w,edgecl,purseonly'], ['BEL-BELMONT PARK', 'May12,2019', '3', '31Mar197AQU6', '1', 'RagtimeSuzy(Castellano,Javier)', '123', 'L', '1', '5', '461/2', '21', '---', '36', '211/4', '2.30', '5wupper,ranon'], ['BEL-BELMONT PARK', 'May12,2019', '3', '2May197BEL6', '8', 'PartyintheSand(Rosario,Joel)', '123', 'L', '5', '1', '5', '41/2', '---', '411', '311/2', '4.60', '6wupper,belatedly'], ['BEL-BELMONT PARK', 'May12,2019', '3', '30Dec186AQU3', '7', 'Athwaaq(Lezcano,Jose)', '123', 'L', '4', '4', '111/2', '11/2', '---', '2Head', '4353/4', '0.60', 'coaxedst,weakened'], ['BEL-BELMONT PARK', 'May12,2019', '3', '13Apr192AQU6', '6', 'AweHoney(Cancel,Eric)', '118', 'L', '3', '2', '311/2', '5', '---', '5', '5', '9.40', 'cha